In [ ]:
from dotenv import load_dotenv
import os
from IPython.display import display, Markdown, Latex
from langchain_openai import ChatOpenAI
from crewai_tools import BaseTool
from embedchain import App

load_dotenv()

model_name = os.getenv("OPENAI_MODEL_NAME", "gpt-4o-mini")
embedding_model_name = os.getenv("LOCAL_EMBEDDINGS_MODEL")

llm = ChatOpenAI(temperature=0, model_name=model_name)

In [ ]:
class SearchInFiles(BaseTool):
    name: str = "Pesquisar no diretório"
    description: str = (
        "Retorna os dados dos arquivos do diretório."
    )

    def _run(self, topic: str) -> str:
        # load chroma configuration from yaml file
        app = App.from_config(config_path="ingestion.yaml")

        result = app.query(input_query=topic, citations=True, limit=5)
        return result

In [ ]:
result = SearchInFiles().run(topic="""Consultas Mongo: 
# Score """)

In [ ]:
saida = result[0]
Markdown(saida)

In [ ]:
fontes = ""
import os
current_dir = os.getcwd()


for i in range(len(result[1])):
    fontes_temp = result[1][i][1]

    fontes += f"**Fonte:** [{fontes_temp['source']}](<./{fontes_temp['source']}>)\n"
    #fontes += f" | [Abrir pasta](<./{fontes_temp['source'][:fontes_temp['source'].rfind('/')]}>)\n"
    fontes += f"**Score:** {fontes_temp['score']}\n\n"

display(Markdown(fontes))

In [ ]:
from crewai import Agent, Crew, Process, Task

# abrir arquivo yaml como dict
def load_yaml(file_path):
    import yaml
    with open(file_path, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

agents_config = load_yaml("src/master_search/config/agents.yaml")
tasks_config = load_yaml("src/master_search/config/tasks.yaml")

In [ ]:
researcher = Agent(
	config=agents_config['researcher'],
    tools=[directory_tool],
    verbose=True,
    llm=llm,
	allow_delegation=False,
)

research_task = Task(
	config=tasks_config['research_task'],
    agent=researcher,
    verbose=True,
    llm=llm,
)


crew = Crew(
  agents=[researcher],
  tasks=[research_task],
  verbose=2,
  memory=False,
  llm=llm,
  full_output=True,
  planning=False,
)

input_text = "Qual query do mongo é usada para buscar os compromissos?"

inputs = {
    "topic": input_text
}
result = crew.kickoff(inputs=inputs)

In [ ]:
Markdown(result.raw)